In [1]:
import pandas as pd
from load_data import load_data
pd.set_option('display.max_columns', None)
import numpy as np
from scipy.stats import ttest_ind
from scipy.stats import kruskal

In [2]:
price, listings, reviews = load_data()

/Users/dmnk/OneDrive - stud.uni-goettingen.de/Dokumente/3. Semester/SeminarDL/DubAir/load_data.py:47: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  price = price.str.replace("$","")


# CATEGORICAL VARIABLES

In [3]:
c = ["room_type", "property_type", "host_location", "host_is_superhost", "host_identity_verified", "neighbourhood_cleansed", "bathrooms_text", "has_availability", "instant_bookable"]
list_cat = listings.filter(c)

# Inspect columns

First we can have a look at the amount of NAs per feature

In [4]:
list_cat.isna().sum()

room_type                  0
property_type              0
host_location             32
host_is_superhost         11
host_identity_verified    11
neighbourhood_cleansed     0
bathrooms_text             4
has_availability           0
instant_bookable           0
dtype: int64

There are only a few missing values for the host_location, host_is_superhost and host_identity_verfied. We will just drop them for the moment.

In [5]:
na_filter = list_cat.isna().any(axis = 1)
price = price[~na_filter]
list_cat = list_cat.dropna()
list_cat.isna().sum()


room_type                 0
property_type             0
host_location             0
host_is_superhost         0
host_identity_verified    0
neighbourhood_cleansed    0
bathrooms_text            0
has_availability          0
instant_bookable          0
dtype: int64

## Lets have a look at each variable

### property_type

In [6]:
list_cat["property_type"].value_counts()

Entire rental unit                     1575
Private room in rental unit            1389
Private room in residential home       1339
Entire residential home                 924
Entire condominium (condo)              268
Private room in townhouse               240
Entire townhouse                        219
Private room in condominium (condo)     158
Entire serviced apartment                95
Private room in bed and breakfast        88
Shared room in residential home          64
Shared room in rental unit               55
Entire guest suite                       48
Entire cottage                           46
Private room in bungalow                 44
Entire guesthouse                        33
Room in hotel                            29
Entire loft                              28
Entire bungalow                          26
Room in aparthotel                       20
Entire cabin                             19
Private room in guest suite              18
Private room in cottage         

In [7]:
list_cat["property_type"].unique()

array(['Private room in bungalow', 'Private room in residential home',
       'Private room in townhouse', 'Entire rental unit',
       'Entire residential home', 'Private room in rental unit',
       'Entire loft', 'Entire condominium (condo)', 'Entire cottage',
       'Entire guesthouse', 'Entire townhouse', 'Boat',
       'Entire serviced apartment', 'Private room in condominium (condo)',
       'Private room in bed and breakfast', 'Tiny house',
       'Shared room in condominium (condo)', 'Entire guest suite',
       'Entire cabin', 'Private room in loft',
       'Shared room in rental unit', 'Entire bungalow', 'Private room',
       'Private room in villa', 'Private room in cottage',
       'Shared room in loft', 'Entire chalet', 'Castle',
       'Private room in cabin', 'Shared room in residential home',
       'Entire place', 'Private room in chalet', 'Entire villa',
       'Private room in floor', 'Private room in guesthouse',
       'Room in aparthotel', 'Shared room', 'Shared

We already got rid of hotels in the room_type variable however there are still hotel rooms in the data according to property_type

In [8]:
prop = list_cat["property_type"]
filter_prop = prop.str.contains("hotel", case = False)
print(prop[filter_prop].unique())
print(filter_prop.sum())

['Room in aparthotel' 'Room in boutique hotel' 'Room in hotel']
63


We have 63 entries of hotel rooms which are either a room in a aparthotel, boutique hotel or just in a normal hotel. We also get rid of those entries. 

In [9]:
list_cat = list_cat[~filter_prop]
price = price[~filter_prop]

We should do the same for hostel rooms as they aren't private aparments/homes either.

In [10]:
prop = list_cat["property_type"]
filter_prop = prop.str.contains("hostel", case = False)
print(prop[filter_prop].unique())
print(filter_prop.sum())
list_cat = list_cat[~filter_prop]
price = price[~filter_prop]

['Shared room in hostel' 'Private room in hostel']
15


Unfortunately there are still some property types where we don't have a sufficient amount of data for (shown here less than 10). We store these ones in a separate category names "others" for now

In [11]:
values = list_cat["property_type"].value_counts()
values.where(values<=10).dropna()
other_list = values.where(values<=10).dropna().index


In [12]:
prop.filter(other_list)

Series([], Name: property_type, dtype: object)

In [13]:
other_list = values.where(values<=10).dropna().index

for i, j in enumerate(other_list):
    fil = list_cat["property_type"].str.contains(j, case = True, na = False)
    list_cat["property_type"][fil] = "Others"

/var/folders/97/j215pw6x7sq158bvx1ktlhf80000gn/T/ipykernel_22812/3501890788.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  fil = list_cat["property_type"].str.contains(j, case = True, na = False)


In [14]:
list_cat["property_type"].value_counts()

Entire rental unit                     1575
Private room in rental unit            1389
Private room in residential home       1339
Entire residential home                 924
Entire condominium (condo)              268
Private room in townhouse               240
Entire townhouse                        219
Others                                  184
Private room in condominium (condo)     158
Entire serviced apartment                95
Private room in bed and breakfast        88
Entire guest suite                       48
Entire cottage                           46
Private room in bungalow                 44
Entire guesthouse                        33
Entire loft                              28
Entire bungalow                          26
Entire cabin                             19
Private room in guest suite              18
Private room in cottage                  16
Private room in guesthouse               14
Tiny house                               13
Private room in cabin           

### room_type
room_type gives us a more granular view of the kind of room which is rent out

In [15]:
list_cat["room_type"].value_counts()

Private room       3354
Entire home/apt    3323
Shared room         131
Name: room_type, dtype: int64

That looks just fine. 

### host_location

In [16]:
pd.unique(list_cat["host_location"])

array(['Dublin  Ireland', 'Dublin, County Dublin, Ireland',
       'Tralee, County Kerry, Ireland', 'Dublin, Dublin, Ireland',
       'Lucan, Co.Dublin. Ireland', 'County Kildare, Ireland',
       'County Dublin, Ireland', 'Salvador Brazil, and Dublin Ireland',
       'Lucan, County Dublin, Ireland', 'Istanbul, İstanbul, Turkey',
       'Ireland', 'Singapore, Singapore', 'Rathfarnham, Dublin, Ireland',
       'Trim, County Meath, Ireland', 'Dublin, Leinster, Ireland',
       'Dublin 2, County Dublin, Ireland', 'Smithfield, Dublin, Ireland',
       'ireland', 'Dublin 8, County Dublin, Ireland',
       'Sallynoggin, County Dublin, Ireland',
       'Luxembourg, Luxembourg District, Luxembourg',
       'New York, New York, United States', 'London',
       'Dublin 6, County Dublin, Ireland', 'Cupramontana, Marche, Italy',
       'Ranelagh, Dublin, Ireland',
       'Tübingen, Baden-Württemberg, Germany', 'Toronto, Ontario, Canada',
       'Glasnevin, Dublin, Ireland', 'County Cork, Ireland',

There are a lot of different entries for Dublin. Might be reasonable to store them in one category.
Therefore turn all values containing Dublin into one category "Dublin, Ireland"

In [17]:
fil = list_cat["host_location"].str.contains("Dublin|DB", case = False, na = False)
list_cat["host_location"][fil] = "Dublin, Ireland"
list_cat["host_location"]

0                     Dublin, Ireland
2                     Dublin, Ireland
3                     Dublin, Ireland
4                     Dublin, Ireland
5       Tralee, County Kerry, Ireland
                    ...              
6971                  Dublin, Ireland
6972                  Dublin, Ireland
6973                  Dublin, Ireland
6974                  Dublin, Ireland
6975                               IE
Name: host_location, Length: 6808, dtype: object

Could be reasonable to use the home country of the owner. Download csv with all countries.

In [18]:
country_abr = pd.read_csv("https://gist.githubusercontent.com/radcliff/f09c0f88344a7fcef373/raw/2753c482ad091c54b1822288ad2e4811c021d8ec/wikipedia-iso-country-codes.csv")
country_list = list(country_abr.iloc[:,0])
abr_list = list(country_abr.iloc[:,1])

In [19]:
list_cat["host_location_country"] = list_cat["host_location"].copy()

for i in list(country_list):
    fil = list_cat["host_location"].str.contains(i, case = False, na = False)
    list_cat["host_location_country"][fil] = str(i)

for i,j in enumerate(list(abr_list)):
    fil = list_cat["host_location"].str.contains(str(j), case = True, na = False)
    list_cat["host_location_country"][fil] = str(country_list[i])

list_cat["host_location_country"].value_counts()


/var/folders/97/j215pw6x7sq158bvx1ktlhf80000gn/T/ipykernel_22812/2422233082.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  fil = list_cat["host_location"].str.contains(i, case = False, na = False)


Ireland                    5803
United Kingdom              391
France                       94
United States                84
Spain                        64
                           ... 
Greece                        1
South Africa                  1
Prague, Prague, Czechia       1
Colombia                      1
Albania                       1
Name: host_location_country, Length: 63, dtype: int64

Surprisingly most of the hosts come from ireland. However we still have a lot of different categories/countries.

In [20]:
other_filter = list_cat["host_location_country"].value_counts() <= 5
other_list = list(list_cat["host_location_country"].value_counts().index[other_filter])

for i, j in enumerate(other_list):
    fil = list_cat["host_location_country"].str.contains(j, case = True, na = False)
    list_cat["host_location_country"][fil] = "Others"

In [21]:
list_cat["host_location_country"].value_counts()

Ireland           5803
United Kingdom     391
France              94
United States       84
Others              72
Spain               64
India               48
Germany             40
Italy               32
Brazil              32
Australia           27
Canada              19
Netherlands         15
Israel              13
Switzerland         12
Sweden              10
Turkey               9
Portugal             7
Argentina            6
Mexico               6
Poland               6
Belgium              6
Bulgaria             6
Croatia              6
Name: host_location_country, dtype: int64

Looking up "53.357852, -6.259787" on google maps leads to an address in Dublin

In [22]:
list_cat["host_location_country"][list_cat["host_location_country"] == "53.357852, -6.259787"] = "Ireland"

### host_is_superhost

In [23]:
list_cat["host_is_superhost"].value_counts()

f    5626
t    1182
Name: host_is_superhost, dtype: int64

seems fine

### host_identity_verified

In [24]:
list_cat["host_identity_verified"].value_counts()

t    4821
f    1987
Name: host_identity_verified, dtype: int64

seems fine too

### neighbourhood_cleansed

In [25]:
list_cat["neighbourhood_cleansed"].value_counts()

Dublin City              5183
Dn Laoghaire-Rathdown     726
Fingal                    623
South Dublin              276
Name: neighbourhood_cleansed, dtype: int64

perfect

### bathrooms_text

In [26]:
list_cat["bathrooms_text"].value_counts()

1 bath               2375
1 shared bath        1276
1 private bath        992
2 baths               770
1.5 baths             425
1.5 shared baths      284
2.5 baths             228
3 baths               125
2 shared baths        118
3.5 baths              49
4 baths                38
2.5 shared baths       28
Private half-bath      18
0 shared baths         15
0 baths                12
4.5 baths              11
3 shared baths         11
Half-bath               7
Shared half-bath        6
5 baths                 5
4 shared baths          4
5.5 baths               3
6 baths                 2
3.5 shared baths        2
7.5 baths               1
6.5 baths               1
8.5 baths               1
9 baths                 1
Name: bathrooms_text, dtype: int64

Some of the categories for the bathroom should be replaced. Half bath can be turned into 0.5 which means just a toilet and a sink without a shower. Then one can differentiate between shared, private (in the room) and a normal bathroom. We construct two variables - one containing the number of bathrooms another the kind of bathrooms howevere "private" or "shared" might indicate that only one of the bathroom is private or shared.

In [27]:
bath = list_cat["bathrooms_text"]
bath_kind = bath.copy()

shared = bath.str.contains("shared", case = False)
private = bath.str.contains("private", case = False)
normal = ~pd.concat([shared, private], axis = 1).any(axis = 1)

bath_kind[shared] = "Shared"
bath_kind[private] = "Private"
bath_kind[normal] = "Normal"

list_cat["bath_kind"] = bath_kind
bath_kind.value_counts()


Normal     4054
Shared     1744
Private    1010
Name: bathrooms_text, dtype: int64

In [28]:
bath_number = bath.copy()
bath_number = bath_number.str.replace("half", "0.5", case = False)
bath_number = bath_number.str.extract('(\d+.\d|\d+)')

list_cat["bath_number"] = bath_number
bath_number.value_counts()

1      4643
2       888
1.5     709
2.5     256
3       136
3.5      51
4        42
0.5      31
0        27
4.5      11
5         5
5.5       3
6         2
6.5       1
7.5       1
8.5       1
9         1
dtype: int64

a lot of different options but everythings seems fine

### has_availability

In [29]:
list_cat["has_availability"].value_counts()

t    6652
f     156
Name: has_availability, dtype: int64

nice

### instant_bookable

In [30]:
list_cat["instant_bookable"].value_counts()

f    4608
t    2200
Name: instant_bookable, dtype: int64

cool cool cool


## EXPLORATION: 
## 1. Binary - t-Test
Let's put every binary variable in a Welch t-Test

In [31]:
stats_val = []
p_val = []
names = []

def t_Test(X, y, stats, p_val, names):
    catg = pd.unique(X)
    catg_filter = (X == catg[0])
    sample1 = y[catg_filter]
    sample2 = y[~catg_filter]
    
    t, p = ttest_ind(sample1, sample2, equal_var = False)
    name = "t: "+X.name

    stats.append(t)
    p_val.append(p)
    names.append(name)


    return t, p

### has_availability

In [32]:
t_Test(list_cat["has_availability"], price, stats_val, p_val, names)


(1.0584513760630865, 0.28988769673338316)

not significant

### instant_bookable

In [33]:
t_Test(list_cat["instant_bookable"], price, stats_val, p_val, names)



(-0.9804557067295888, 0.32696907010692267)

not significant

### host_identity_verified

In [34]:
t_Test(list_cat["host_identity_verified"], price, stats_val, p_val, names)



(-0.9535309105928556, 0.3404371709239449)

not significant 

### host_is_superhost

In [35]:
t_Test(list_cat["host_is_superhost"], price, stats_val, p_val, names)



(-1.006165989777516, 0.314378758980841)

not significant

### host_location_country (Ireland or not)

In [36]:
X = list_cat["host_location_country"].where(list_cat["host_location_country"] == "Ireland", "else")
t_Test(X, price, stats_val, p_val, names)



(0.691817380333088, 0.48907955258692104)

not significant

### 2. multinomial - ANOVA 
so lets find out whether one of the multicategorical variables have at least one group that is significantly different from the others. Therefore we don't use the common ANOVA as the assumption of equal variances across groups is usually not met. Instead we use the Kruskal-Test which is implemented in scipy

### host_location_country

In [37]:
def krus_test(X, y, stats, p_val, names):
    c_list = pd.unique(X)

    F, p = kruskal(*[list(y[X == i]) for i in c_list])
    name = "F: " + X.name

    stats.append(F)
    p_val.append(p)
    names.append(name)

    return F, p

In [38]:
krus_test(list_cat["host_location_country"], price, stats_val, p_val, names)


(88.35906244473792, 1.338127159451867e-09)

significant

### bath_kind

In [39]:
krus_test(list_cat["bath_kind"], price, stats_val, p_val, names)


(2035.4688645990084, 0.0)

significant

### neighbourhood_cleansed

In [40]:
stat, p = krus_test(list_cat["neighbourhood_cleansed"], price, stats_val, p_val, names)


significant

### property_type

In [41]:
stat, p = krus_test(list_cat["property_type"], price, stats_val, p_val, names)


significant

### room_type

In [42]:
stat, p = krus_test(list_cat["room_type"], price, stats_val, p_val, names)


significant. Is it reasonable to include both property_type and room_type? We test for the interaction

In [43]:
types = list_cat["property_type"] + list_cat["room_type"]

types.name = "Interaction_room_prop"

stat, p = krus_test(types, price, stats_val, p_val, names)


# Summary

In [44]:
d = {"Variable": names, "t-/F-Statistic": stats_val, "p-value": p_val}
pd.DataFrame(d)

,Variable,t-/F-Statistic,p-value
0,t: has_availability,1.058451,2.898877e-01
1,t: instant_bookable,-0.980456,3.269691e-01
2,t: host_identity_verified,-0.953531,3.404372e-01
3,t: host_is_superhost,-1.006166,3.143788e-01
4,t: host_location_country,0.691817,4.890796e-01
5,F: host_location_country,88.359062,1.338127e-09
6,F: bath_kind,2035.468865,0.000000e+00
7,F: neighbourhood_cleansed,131.938539,2.066765e-28
8,F: property_type,3108.147162,0.000000e+00
9,F: room_type,2923.516301,0.000000e+00


the multicategorical variables seem to explain some of the variance in price and should be taken into account 

#

# AMENITIES

In [3]:
import pandas as pd
from load_data import load_data
pd.set_option('display.max_columns', None)
import numpy as np
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import seaborn as sn
from scipy.stats import ttest_ind
from scipy.stats import kruskal
import helpers
from load_data import *

In [4]:
price, listings, reviews = load()

Data loaded.


In [5]:
amenities = listings["amenities"]

In [6]:
type(amenities[0])

str

Problem: The amenities are not stored in a list for each observation but are a string of a list representation 

In [7]:
amenities = amenities.apply(ast.literal_eval)
type(amenities[0])

list

The elements are now of type list.

Next step: turn them into a list of dummy-variables which indicate wheter an apartment has this particular amenity or not

In [8]:
mlb = MultiLabelBinarizer()
am_array = mlb.fit_transform(amenities)

am_df = pd.DataFrame(am_array, columns = mlb.classes_)
am_df.head()

,2-ring mini hob electric stove,2-ring mini hob. electric stove,"22"" HDTV with Amazon Prime Video, Netflix","22"" TV","24"" TV","27"" TV","28"" HDTV","32"" HDTV","32"" HDTV with Amazon Prime Video, Apple TV, Netflix","32"" HDTV with Amazon Prime Video, Netflix, standard cable","32"" HDTV with Netflix, premium cable","32"" HDTV with Netflix, standard cable","32"" HDTV with standard cable","32"" HDTV with standard cable, Netflix, premium cable","32"" TV","32"" TV with Chromecast","32"" TV with Netflix, standard cable","32"" TV with standard cable, premium cable, Netflix","36"" TV with standard cable","38"" TV with premium cable, Netflix, standard cable","40"" HDTV","40"" HDTV with Amazon Prime Video, Netflix, premium cable, standard cable","40"" HDTV with Amazon Prime Video, premium cable, Netflix","40"" HDTV with Chromecast, standard cable","40"" HDTV with Netflix, premium cable","40"" HDTV with Netflix, standard cable","40"" HDTV with standard cable","40"" TV","42"" HDTV","42"" HDTV with Chromecast, Netflix, standard cable","42"" HDTV with Netflix","42"" HDTV with Netflix, Chromecast","42"" HDTV with Netflix, standard cable","42"" HDTV with Netflix, standard cable, Chromecast, premium cable","42"" TV with Netflix, premium cable","42"" TV with standard cable, Amazon Prime Video, Netflix, premium cable","43"" HDTV with Chromecast, Netflix, standard cable","43"" HDTV with premium cable","43"" HDTV with standard cable","45"" HDTV with Netflix, Amazon Prime Video","48"" HDTV with Netflix, premium cable","49"" HDTV with Amazon Prime Video, Netflix, premium cable, standard cable","50"" HDTV","50"" HDTV with Chromecast, Netflix","50"" HDTV with Netflix","50"" TV with Chromecast","50"" TV with Netflix","50"" TV with premium cable","52"" HDTV with Netflix","55"" HDTV","55"" HDTV with","55"" HDTV with Amazon Prime Video, Netflix","60"" HDTV with Netflix","62"" HDTV with Netflix","65"" HDTV","65"" HDTV with Amazon Prime Video","65"" HDTV with Amazon Prime Video, Chromecast, Netflix","75"" HDTV with Netflix, Amazon Prime Video, Apple TV, Roku",? refrigerator,AEG electric stove,AEG oven,AEG refrigerator,Air conditioning,Alexa Amazon Bluetooth sound system,Aragan oil conditioner,BBQ grill,Baby bath,Baby monitor,Baby safety gates,Babysitter recommendations,Backyard,Baking sheet,Bang & Olufsen sound system with Bluetooth and aux,Barbecue utensils,Basic oven,Basic stove,Bathroom essentials,Bathtub,Beach essentials,Beachfront,Bed linens,Bedroom comforts,Beko electric stove,Beko oven,Bellina electric stove,Belling 4 ring hob stove,Belling stainless steel oven,Belo refrigerator,Bidet,Bikes,Bluetooth sound system,Board games,Boat slip,Boche oven,Body soap,Bora stove,Bosch electric stove,Bosch refrigerator,Bosch stainless steel oven,Bose sound system,Bose Bluetooth sound system,Bread maker,Breakfast,Building staff,Built in wall sound system sound system with aux,Cable TV,Carbon monoxide alarm,Ceiling fan,Central air conditioning,Central heating,Changing table,Children’s books and toys,Children’s books and toys for ages 0-2 years old,Children’s books and toys for ages 0-2 years old and 2-5 years old,"Children’s books and toys for ages 0-2 years old, 2-5 years old, 5-10 years old, and 10+ years old","Children’s books and toys for ages 0-2 years old, 2-5 years old, and 5-10 years old",Children’s books and toys for ages 2-5 years old,Children’s books and toys for ages 2-5 years old and 5-10 years old,Children’s books and toys for ages 5-10 years old and 10+ years old,Children’s dinnerware,Cleaning before checkout,Cleaning products,Clothing storage,Clothing storage: closet,Clothing storage: closet and dresser,Clothing storage: closet and wardrobe,"Clothing storage: closet, dresser, and wardrobe","Clothing storage: closet, wardrobe, and dresser",Clothing storage: dresser,Clothing storage: dresser and closet,Clothing storage: dresser and wardrobe,"Clothing storage: dresser, closet, and wardrobe","Clothing storage: dresser, wardrobe, and closet",Clothing s

In [9]:
am_df.shape

(6922, 520)

We could try to summarize some of them into categories. 
- all HDTVs and TVs into a new variable containing the absolute number of TVs in the apartment
- all children's books and toys to one variable which indicates whether there is children's entertainment or not
- all variables about a dedicated workspace to one variable indicating whether there is a dedicated workspace or not
- all different brands of shampoo, conditioner, stove, ofen, refrigerator, body soap, sound system, coffee machine, 
- all different variables containing information about the fast wifi speed into one
- all free parking/free carport/free garage variables into one which gives the total number of free parking spaces
- all paid parking variables into one which gives the total number of paid parking spaces
- all HDTV variables into one
- all free washer variables into one
- all game console variables into one
- all dryer variables into one
- paid waher into one
- garden or backyard into one
- gym into one

Also we could get rid of some variables which might not have the greatest impact on the price like 
- all different clothing storages
- hangers 
- hot water kattle


Okay lets start with the summaries. Let's write a function which deals with it

In [10]:
def in_one(df, in_cat, out_cat_name, regex = True, sum = False, drop = True):

    if regex == True:
        # get columns to substitute
        col_filter = df.columns.str.contains(in_cat, case = False, regex = True)
        col_names = df.columns[col_filter]

        # create new column
        if sum == True:
            new_var = df.filter(col_names).sum(axis = 1)
        else:
            new_var = df.filter(col_names).sum(axis = 1)
            new_var[new_var>1] = 1
    
    else:
        col_names = in_cat
        if sum == True:
            new_var = df.filter(col_names).sum(axis = 1)
        else:
            new_var = df.filter(col_names).sum(axis = 1)
            new_var[new_var>1] = 1
    df[out_cat_name] = new_var

    if drop == True:
        df = df.drop(col_names, axis = 1)
    
    return df

In [11]:
def add_col(df, sender, receiver, regex = True):
    if regex == True:
        col_filter = df.columns.str.contains(sender, case = False, regex = True)
        sender = df.columns[col_filter] 

    df[receiver] = df.filter(sender).sum(axis=1)+df[receiver]
    df = df.drop(sender, axis = 1)
    return df

In [12]:
def drop_col(df, sender, regex = True):
    if regex == True:
        col_filter = df.columns.str.contains(sender, case = False, regex = True)
        sender = df.columns[col_filter] 
    df = df.drop(sender, axis = 1)
    return df

In [13]:
am_df

,2-ring mini hob electric stove,2-ring mini hob. electric stove,"22"" HDTV with Amazon Prime Video, Netflix","22"" TV","24"" TV","27"" TV","28"" HDTV","32"" HDTV","32"" HDTV with Amazon Prime Video, Apple TV, Netflix","32"" HDTV with Amazon Prime Video, Netflix, standard cable","32"" HDTV with Netflix, premium cable","32"" HDTV with Netflix, standard cable","32"" HDTV with standard cable","32"" HDTV with standard cable, Netflix, premium cable","32"" TV","32"" TV with Chromecast","32"" TV with Netflix, standard cable","32"" TV with standard cable, premium cable, Netflix","36"" TV with standard cable","38"" TV with premium cable, Netflix, standard cable","40"" HDTV","40"" HDTV with Amazon Prime Video, Netflix, premium cable, standard cable","40"" HDTV with Amazon Prime Video, premium cable, Netflix","40"" HDTV with Chromecast, standard cable","40"" HDTV with Netflix, premium cable","40"" HDTV with Netflix, standard cable","40"" HDTV with standard cable","40"" TV","42"" HDTV","42"" HDTV with Chromecast, Netflix, standard cable","42"" HDTV with Netflix","42"" HDTV with Netflix, Chromecast","42"" HDTV with Netflix, standard cable","42"" HDTV with Netflix, standard cable, Chromecast, premium cable","42"" TV with Netflix, premium cable","42"" TV with standard cable, Amazon Prime Video, Netflix, premium cable","43"" HDTV with Chromecast, Netflix, standard cable","43"" HDTV with premium cable","43"" HDTV with standard cable","45"" HDTV with Netflix, Amazon Prime Video","48"" HDTV with Netflix, premium cable","49"" HDTV with Amazon Prime Video, Netflix, premium cable, standard cable","50"" HDTV","50"" HDTV with Chromecast, Netflix","50"" HDTV with Netflix","50"" TV with Chromecast","50"" TV with Netflix","50"" TV with premium cable","52"" HDTV with Netflix","55"" HDTV","55"" HDTV with","55"" HDTV with Amazon Prime Video, Netflix","60"" HDTV with Netflix","62"" HDTV with Netflix","65"" HDTV","65"" HDTV with Amazon Prime Video","65"" HDTV with Amazon Prime Video, Chromecast, Netflix","75"" HDTV with Netflix, Amazon Prime Video, Apple TV, Roku",? refrigerator,AEG electric stove,AEG oven,AEG refrigerator,Air conditioning,Alexa Amazon Bluetooth sound system,Aragan oil conditioner,BBQ grill,Baby bath,Baby monitor,Baby safety gates,Babysitter recommendations,Backyard,Baking sheet,Bang & Olufsen sound system with Bluetooth and aux,Barbecue utensils,Basic oven,Basic stove,Bathroom essentials,Bathtub,Beach essentials,Beachfront,Bed linens,Bedroom comforts,Beko electric stove,Beko oven,Bellina electric stove,Belling 4 ring hob stove,Belling stainless steel oven,Belo refrigerator,Bidet,Bikes,Bluetooth sound system,Board games,Boat slip,Boche oven,Body soap,Bora stove,Bosch electric stove,Bosch refrigerator,Bosch stainless steel oven,Bose sound system,Bose Bluetooth sound system,Bread maker,Breakfast,Building staff,Built in wall sound system sound system with aux,Cable TV,Carbon monoxide alarm,Ceiling fan,Central air conditioning,Central heating,Changing table,Children’s books and toys,Children’s books and toys for ages 0-2 years old,Children’s books and toys for ages 0-2 years old and 2-5 years old,"Children’s books and toys for ages 0-2 years old, 2-5 years old, 5-10 years old, and 10+ years old","Children’s books and toys for ages 0-2 years old, 2-5 years old, and 5-10 years old",Children’s books and toys for ages 2-5 years old,Children’s books and toys for ages 2-5 years old and 5-10 years old,Children’s books and toys for ages 5-10 years old and 10+ years old,Children’s dinnerware,Cleaning before checkout,Cleaning products,Clothing storage,Clothing storage: closet,Clothing storage: closet and dresser,Clothing storage: closet and wardrobe,"Clothing storage: closet, dresser, and wardrobe","Clothing storage: closet, wardrobe, and dresser",Clothing storage: dresser,Clothing storage: dresser and closet,Clothing storage: dresser and wardrobe,"Clothing storage: dresser, closet, and wardrobe","Clothing storage: dresser, wardrobe, and closet",Clothing s

In [14]:
# first drop stuff we don't need anymore (too standard, too broad, too specific)
am_df = drop_col(am_df, "(Clothing storage)")
am_df = drop_col(am_df, ["Bedroom comforts", "Bread maker","Carbon monoxide alarm",
"Children’s dinnerware", "Drying rack for clothing", "Fireplace guards", "Fire extinguisher", 
"Hot water kettle", "Hangers", "Iron", "Keypad", "Pocket wifi", "Mini fridge",
"Mosquito net", "Outlet covers", "Pour-over coffee", "Portable fans",
"Portable heater", "Portable air conditioning", "Radiant heating", "Record player", 
"Rice maker", "Shower gel", "Ski-in/Ski-out", "Table corner guards", "Trash compactor",
"Wine glasses", "Window guards", "Baking sheet", "Barbecue utensils", "Boat slip",
"Cable TV","Changing table","Cleaning products","EV charger","Ethernet connection", 
"Extra pillows and blankets", "First aid kit","Laundromat nearby", "Room-darkening shades",
"Smart lock", "Smoke alarm", "Toaster", "Microwave", "Essentials", "Bathroom essentials", "Fire pit", 
"Lock on bedroom door", "Hot water", "Beach essentials", "Board games", "Building staff", 
"Cooking basics", "Dining table", "Dishes and silverware", "Host greets you", "Luggage dropoff allowed", 
"Self check-in"], regex = False)

# sum up all luxury or extraordinary equipment
am_df = in_one(am_df, ["Piano", "Ping pong table", "Kayak", "BBQ grill", "Bidet", "Bikes"], "Special_stuff", regex = False, sum = True, drop = True)

# sum true
am_df = in_one(am_df, "(HDTV)|(^\d\d..TV)|(^TV)", "TV_number", regex = True, sum = True, drop = True)
am_df = in_one(am_df, "(game console)", "Oven_number", regex = True, sum = True, drop = True)
am_df = in_one(am_df, "(^outdoor)", "Outdoor_stuff_number", regex = True, sum = True, drop = True)
am_df = in_one(am_df, "(^Baby)|(^Crib$)|( crib$)|(^High chair$)", "Baby_friendly", regex = True, sum = True, drop = True)


# sum false
am_df = in_one(am_df, "(body soap)", "Body_soap_available", regex = True, sum = False, drop = True)
am_df = in_one(am_df, "(.oven)|(^oven)", "Oven_available", regex = True, sum = True, drop = True)
am_df = in_one(am_df, "(garden or backyard)|(^backyard)|(^garden)", "Garden_backyard_available", regex = True, sum = False, drop = True)
am_df = in_one(am_df, "(^free.*parking)|(^free.*garage)|(^free.*carport)", "Free_parking_number", regex = True, sum = False, drop = True)
am_df = in_one(am_df, "(^paid.*parking)|(^paid.*garage)|(^paid.*carport)", "Paid_parking_number", regex = True, sum = False, drop = True)
am_df = in_one(am_df, "(.stove)|(^stove)", "Stoves_number", regex = True, sum = True, drop = True)
am_df = in_one(am_df, "(refrigerator.)|(refrigerator)|(^Freezer$)", "Refridgerator_number", regex = True, sum = True, drop = True)
am_df = in_one(am_df, "(sound system)", "sound_system_number", regex = True, sum = True, drop = True)
am_df = in_one(am_df, "(^Children’s books and toys)", "Children_Entertainment", regex = True, sum = False, drop = True)
am_df = in_one(am_df, "(^Dedicated workspace)", "Workspace", regex = True, sum = False, drop = True)
am_df = in_one(am_df, "(conditioner)|(shampoo)", "Shampoo_Conditioner_available", regex = True, sum = False, drop = True)
am_df = in_one(am_df, "(^Fast wifi.)", "Fast_wifi_available", regex = True, sum = False, drop = True)
am_df = in_one(am_df, "(^Gym)|(. gym)", "Gym_available", regex = True, sum = False, drop = True)
am_df = in_one(am_df, "(coffee machine)|(Nespresso)|(^Coffee maker)", "Coffee_machine_available", regex = True, sum = False, drop = True)
am_df = in_one(am_df, "(^Dryer)|(Paid dryer)|(^Free dryer)", "Dryer_available", regex = True, sum = False, drop = True)
am_df = in_one(am_df, "(^Washer)|(Paid washer)|(Free washer)", "Washer_available", regex = True, sum = False, drop = True)
am_df = in_one(am_df, "(^Hot tub)|(.hot tub)", "Hot_tub_available", regex = True, sum = False, drop = True)
am_df = in_one(am_df, "(^Pool)|(shared.*pool)|(private.*pool)", "Pool_available", regex = True, sum = False, drop = True)
am_df = in_one(am_df, "(patio or balcony)", "Patio_balcony_available", regex = True, sum = False, drop = True)
am_df = in_one(am_df, "(sauna)", "Sauna_available", regex = True, sum = False, drop = True)
am_df = in_one(am_df, "(^Wifi)", "Wifi_available", regex = True, sum = False, drop = True)
am_df = in_one(am_df, "(air conditioning)", "AC_available", regex = True, sum = False, drop = True)
am_df = in_one(am_df, "(heating)", "heating_available", regex = True, sum = False, drop = True)
am_df = in_one(am_df, "(^Kitchen$)|(^Full kitchen$)", "kitchen_available", regex = True, sum = False, drop = True)
am_df = in_one(am_df, "(^Lockbox$)|(^Safe$)", "Safe_available", regex = True, sum = False, drop = True)
am_df = in_one(am_df, "(^Waterfront$)|(^Beachfront$)|(^Lake access$)", "Water_location", regex = True, sum = False, drop = True)




In [15]:
am_df.head()


,Bathtub,Bed linens,Breakfast,Ceiling fan,Cleaning before checkout,Dishwasher,Elevator,Hair dryer,Indoor fireplace,Long term stays allowed,Pets allowed,Private entrance,Security cameras on property,Single level home,Suitable for events,Special_stuff,TV_number,Outdoor_stuff_number,Baby_friendly,Body_soap_available,Oven_available,Garden_backyard_available,Free_parking_number,Paid_parking_number,Stoves_number,Refridgerator_number,sound_system_number,Children_Entertainment,Workspace,Shampoo_Conditioner_available,Fast_wifi_available,Gym_available,Coffee_machine_available,Dryer_available,Washer_available,Hot_tub_available,Pool_available,Patio_balcony_available,Sauna_available,Wifi_available,AC_available,heating_available,kitchen_available,Safe_available,Water_location
0,1,1,1,0,0,1,0,1,1,1,0,0,0,1,0,0,0,2,0,1,1,1,1,0,0,2,0,0,1,1,0,0,1,1,1,0,0,1,0,1,1,1,0,0,0
1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,1,1,0,0
2,0,1,1,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,1,0,0,1,1,1,0,0,1,0,1,0,1,1,0,0
3,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,1,1,0,1,1,0,0,1,1,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0
4,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,4,1,1,0,1,0,1,2,1,0,1,1,0,1,1,1,1,0,0,0,0,1,0,1,1,1,0


In [16]:
len(pd.unique(am_df.columns))

45

We still have more than 50 different amenities.
Let's have a look how man of these are observed frequently

In [17]:
summary = am_df[am_df !=0].sum(axis = 0).sort_values(ascending = False)
summary.where(summary <=20).dropna()

Fast_wifi_available    11.0
Sauna_available         8.0
Ceiling fan             7.0
Suitable for events     1.0
Pets allowed            1.0
dtype: float64

 The amenities seem to be less relevant, moreover it seems strange that only 1 listing allows pets and only 1 is suitable for events. Also fast wifi might be something that is quite subjective. A ceiling fan might not be the most important feature of an apartment in Dublin. A sauna however is quite luxurious. Lets add it to our special_stuff variable

In [18]:
am_df["Special_stuff"] = am_df["Special_stuff"] + am_df["Sauna_available"]
drop_am = list(summary.where(summary <=15).dropna().index)
am_df = drop_col(am_df, drop_am, regex = False)


In [19]:
am_df.describe()

,Bathtub,Bed linens,Breakfast,Cleaning before checkout,Dishwasher,Elevator,Hair dryer,Indoor fireplace,Long term stays allowed,Private entrance,Security cameras on property,Single level home,Special_stuff,TV_number,Outdoor_stuff_number,Baby_friendly,Body_soap_available,Oven_available,Garden_backyard_available,Free_parking_number,Paid_parking_number,Stoves_number,Refridgerator_number,sound_system_number,Children_Entertainment,Workspace,Shampoo_Conditioner_available,Gym_available,Coffee_machine_available,Dryer_available,Washer_available,Hot_tub_available,Pool_available,Patio_balcony_available,Wifi_available,AC_available,heating_available,kitchen_available,Safe_available,Water_location
count,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000,6922.000000
mean,0.101271,0.369402,0.184484,0.021237,0.223490,0.234325,0.670904,0.224213,0.726813,0.259896,0.065444,0.060965,0.059376,0.621208,0.070789,0.227535,0.055042,0.367235,0.194597,0.526293,0.138399,0.279399,0.463883,0.010691,0.050852,0.571078,0.679428,0.039728,0.238081,0.476741,0.756718,0.028460,0.004190,0.180006,0.934268,0.039006,0.930945,0.852499,0.146201,0.026871
std,0.301709,0.482678,0.387907,0.144183,0.416614,0.423607,0.469919,0.417093,0.445628,0.438609,0.247325,0.239283,0.264063,0.485121,0.335991,0.768983,0.228078,0.510614,0.395919,0.499344,0.345344,0.448736,0.599748,0.102848,0.219712,0.494958,0.466730,0.195335,0.425940,0.499495,0.429095,0.166295,0.064596,0.384220,0.247832,0.193623,0.253566,0.354630,0.353333,0.161718
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000
75%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,6.000000,1.000000,2.000000,7.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
am_df.shape

(6922, 40)

# ADDED TO load_data_cleansed FUNCTION !!!